In [78]:
# Wind Capacity Prediction

In [79]:
import numpy as np
import pandas as pd
import xarray as xr
from enum import Enum, auto
from shapely.geometry import Point
import matplotlib.pyplot as plt

paths = {"era5_eu_2013": "resources/europe-2013-era5.nc",
         "era5_tutorial": "resources/europe-2013-era5-tutorial.nc",
         "offshore_shape": "resources/regions_offshore_elec_s_37.geojson",
         "onshore_shape": "resources/regions_onshore_elec_s_37.geojson",
         "capfacs": "resources/capfacs_37.csv",
         "era5_regions": "resources/europe-2013-era5-regions.nc"}

In [80]:
ds = xr.open_dataset(filename_or_obj=paths["era5_regions"], engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:           (region: 65, time: 8760)
Coordinates:
  * region            (region) object 'AL0 0 on' 'AT0 0 on' ... 'SI0 0 off'
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    height            (region, time) float64 ...
    wnd100m           (region, time) float64 ...
    roughness         (region, time) float64 ...
    influx_toa        (region, time) float64 ...
    influx_direct     (region, time) float64 ...
    influx_diffuse    (region, time) float64 ...
    albedo            (region, time) float64 ...
    temperature       (region, time) float64 ...
    soil_temperature  (region, time) float64 ...
    runoff            (region, time) float64 ...
Attributes:
    description:  Era5 data with mean value of the coordinates within a region

In [81]:
capfacts = pd.read_csv(paths["capfacs"])
capfacts

,snapshot,AL0 0 offwind-ac,AL0 0 onwind,AL0 0 solar,AT0 0 onwind,AT0 0 ror,AT0 0 solar,BA0 0 onwind,BA0 0 solar,BE0 0 offwind-ac,...,SE4 0 onwind,SE4 0 ror,SE4 0 solar,SI0 0 offwind-ac,SI0 0 onwind,SI0 0 ror,SI0 0 solar,SK0 0 onwind,SK0 0 ror,SK0 0 solar
0,2013-01-01 00:00:00,0.003291,0.001469,0.0,0.163262,0.224456,0.0,0.007340,0.0,1.000000,...,0.459609,0.626955,0.0,0.000000,0.055146,0.344668,0.0,0.361009,0.106197,0.0
1,2013-01-01 01:00:00,0.002103,0.000000,0.0,0.171340,0.224369,0.0,0.007939,0.0,0.999998,...,0.463265,0.625502,0.0,0.000000,0.052605,0.344657,0.0,0.368912,0.106012,0.0
2,2013-01-01 02:00:00,0.000000,0.000000,0.0,0.171035,0.224300,0.0,0.007829,0.0,0.993941,...,0.463777,0.624810,0.0,0.000000,0.052222,0.344593,0.0,0.382949,0.105968,0.0
3,2013-01-01 03:00:00,0.000000,0.000000,0.0,0.169685,0.224249,0.0,0.005766,0.0,0.916094,...,0.463041,0.623794,0.0,0.000000,0.050762,0.344626,0.0,0.388344,0.106215,0.0
4,2013-01-01 04:00:00,0.000000,0.000000,0.0,0.159757,0.224213,0.0,0.004262,0.0,0.704786,...,0.457253,0.623085,0.0,0.000000,0.047285,0.344607,0.0,0.409303,0.106364,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,0.029235,0.024321,0.0,0.109127,0.275353,0.0,0.001076,0.0,0.998922,...,0.358191,0.469586,0.0,0.027877,0.001708,0.696975,0.0,0.125735,0.157729,0.0
8756,2013-12-31 20:00:00,0.034024,0.031357,0.0,0.120485,0.275159,0.0,0.001090,0.0,0.965757,...,0.324709,0.469395,0.0,0.023567,0.000000,0.696262,0.0,0.142431,0.157817,0.0
8757,2013-12-31 21:00:00,0.037104,0.034522,0.0,0.128186,0.274975,0.0,0.001062,0.0,0.852484,...,0.288415,0.469208,0.0,0.000000,0.004891,0.695524,0.0,0.153930,0.157899,0.0
8758,2013-12-31 22:00:00,0.029730,0.027419,0.0,0.124740,0.274784,0.0,0.001351,0.0,0.776442,...,0.268695,0.469028,0.0,0.000000,0.009091,0.694665,0.0,0.163442,0.157945,0.0


In [82]:
capfacts_dk = capfacts.filter(regex="DK.*")

## Countries for optimizing hyperparameters

Countries with the highest [installed wind power capacity](https://en.wikipedia.org/wiki/Wind_power_in_the_European_Union) in 2021:
1. Germany
2. Spain
3. UK
4. France
5. Sweden

Countries where wind covers the highest [percentage of the electricity demand](https://windeurope.org/wp-content/uploads/files/about-wind/statistics/WindEurope-Annual-Statistics-2019.pdf) in 2019:
1. Denmark
2. Ireland
3. Portugal
4. Germany
5. UK
6. Spain

In [83]:
# ds["region"].values


### Denmark (DK)

- 08 DK0 0 on: 48 points
- 09 DK4 0 on: 10 points
- 41 DK0 0 off: 135 points
- 42 DK4 0 off: 38 points

In [84]:
ds.sel(region="DK0 0 on")

<xarray.Dataset>
Dimensions:           (time: 8760)
Coordinates:
    region            <U8 'DK0 0 on'
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    height            (time) float64 ...
    wnd100m           (time) float64 ...
    roughness         (time) float64 ...
    influx_toa        (time) float64 ...
    influx_direct     (time) float64 ...
    influx_diffuse    (time) float64 ...
    albedo            (time) float64 ...
    temperature       (time) float64 ...
    soil_temperature  (time) float64 ...
    runoff            (time) float64 ...
Attributes:
    description:  Era5 data with mean value of the coordinates within a region

In [85]:
class EnergyType(Enum):
    OFFWIND_AC = auto()
    OFFWIND_DC = auto()
    ONWIND = auto()
    SOLAR = auto()
    ROR = auto()
    NOT_DEFINED = auto()

def get_energy_type(name: str) -> EnergyType:
    match name:
        case "offwind-ac":
            return EnergyType.OFFWIND_AC
        case "offwind-dc":
            return EnergyType.OFFWIND_DC
        case "onwind-dc":
            return EnergyType.ONWIND
        case "solar":
            return EnergyType.SOLAR
        case "ror":
            return EnergyType.ROR
        case _:
            return EnergyType.NOT_DEFINED

In [86]:
def parse_capfac_col(column_name: str) -> (str, EnergyType):
    col_args = column_name.split(" ")
    if len(col_args) == 3:
        region_name = col_args[0]
        energy_type = get_energy_type(col_args[2])
        return region_name, energy_type
    return None

In [87]:
region_name = "DK0 0 on"

X_height = ds.sel(region=region_name)["height"].values
X_wnd100m = ds.sel(region=region_name)["wnd100m"].values
X_roughness = ds.sel(region=region_name)["roughness"].values
X_influx_toa = ds.sel(region=region_name)["influx_toa"].values
X_influx_direct = ds.sel(region=region_name)["influx_direct"].values
X_influx_diffuse = ds.sel(region=region_name)["influx_diffuse"].values
X_albedo = ds.sel(region=region_name)["albedo"].values
X_temperature = ds.sel(region=region_name)["temperature"].values
X_soil_temperature = ds.sel(region=region_name)["soil_temperature"].values
X_runoff = ds.sel(region=region_name)["runoff"].values

In [88]:
for column in capfacts:
    # print(column)
    print(parse_capfac_col(column))

None
('AL0', <EnergyType.OFFWIND_AC: 1>)
('AL0', <EnergyType.NOT_DEFINED: 6>)
('AL0', <EnergyType.SOLAR: 4>)
('AT0', <EnergyType.NOT_DEFINED: 6>)
('AT0', <EnergyType.ROR: 5>)
('AT0', <EnergyType.SOLAR: 4>)
('BA0', <EnergyType.NOT_DEFINED: 6>)
('BA0', <EnergyType.SOLAR: 4>)
('BE0', <EnergyType.OFFWIND_AC: 1>)
('BE0', <EnergyType.OFFWIND_DC: 2>)
('BE0', <EnergyType.NOT_DEFINED: 6>)
('BE0', <EnergyType.ROR: 5>)
('BE0', <EnergyType.SOLAR: 4>)
('BG0', <EnergyType.OFFWIND_AC: 1>)
('BG0', <EnergyType.OFFWIND_DC: 2>)
('BG0', <EnergyType.NOT_DEFINED: 6>)
('BG0', <EnergyType.ROR: 5>)
('BG0', <EnergyType.SOLAR: 4>)
('CH0', <EnergyType.NOT_DEFINED: 6>)
('CH0', <EnergyType.ROR: 5>)
('CH0', <EnergyType.SOLAR: 4>)
('CZ0', <EnergyType.NOT_DEFINED: 6>)
('CZ0', <EnergyType.ROR: 5>)
('CZ0', <EnergyType.SOLAR: 4>)
('DE0', <EnergyType.OFFWIND_AC: 1>)
('DE0', <EnergyType.OFFWIND_DC: 2>)
('DE0', <EnergyType.NOT_DEFINED: 6>)
('DE0', <EnergyType.ROR: 5>)
('DE0', <EnergyType.SOLAR: 4>)
('DK0', <EnergyType.OFFWI